In [103]:
import pandas as pd

df = pd.read_csv('UL-Standards-List.csv')
# drop rows with no description
df = df.dropna(subset=['Description'])
len(df)
# df.head()


2123

In [104]:
# Description to Title Case since there are some descriptions that are all caps
df['Description'] = df['Description'].str.title()





In [105]:

df.iloc[144]['Description']

'Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method) / Standard Method Of Test For Ignition Resistance Of Loose Fill Insulation (Cigarette Method)'

In [106]:
# Removing duplicates in the description separated by ' / '
def remove_duplicates(string):
    words = string.split(' / ')
    result = []
    for word in words:
        if word not in result:
            result.append(word)
    return ' / '.join(result)  # nothing to join if there are no duplicates

df['Description'] = df['Description'].apply(remove_duplicates)
df.iloc[570]['Description']

'Standard For Category 3 Film-Forming Fluoroprotein (Fffp) Foam Liquid Concentrates'

In [107]:
# Look for non-English descriptions
def isEnglish(s):
    return s.isascii()

df['English?'] = df['Description'].apply(isEnglish)


In [108]:
df['Description'] = df['Description'].str.replace(' For ', ' for ')
df['Description'] = df['Description'].str.replace(' Of ', ' of ')

In [109]:
df['Category'] = ''
df['Topic'] = ''

In [110]:
# Remove non-English descriptions
for idx, row in df.iterrows():
    if row['English?'] == False:
        split_description = row['Description'].split(' / ', maxsplit=1)
        if len(split_description) > 1:
            df.at[idx, 'Topic'] = split_description[0]
        else:
            df.at[idx, 'Topic'] = row['Description']
    else:
        df.at[idx, 'Topic'] = row['Description']



In [111]:
df['Topic'][1505]

'Ieee/Ul Wireless Diabetes Device Security: Information Security Requirements for Connected Diabetes Solutions'

In [112]:
import re
def find_words_in_description(string, words):
    start_index = 0
    found_word = ''
    for word in words:
        i = string.find(word, start_index)
        if i != -1:
            if i > 40:
                continue
            else:
                start_index = i + len(word)

            found_word = found_word + ' ' + word
        else:
            start_index = start_index
    return (start_index, found_word)

words = ['Standard', 'Method', 'Test', 'Outline', 'Guide', 'Guidance', 'Criteria', 'Procedure', 'Rule', 'Requirement','Enclosure']


for idx, row in df.iterrows():
    i = find_words_in_description(row['Topic'], words)[0]
    final_word = find_words_in_description(row['Topic'], words)[1]
    print(i, final_word)
    if final_word != '':
        df.at[idx, 'Category'] = final_word
        split_topic_desc = re.split(' for | of ', row['Topic'][i:], maxsplit=1)
        # join the word in description before the split if not in the list of words

        print(split_topic_desc)
        if len(split_topic_desc) > 1:
            lost_words = [word for word in row['Topic'][:i] if word not in words]
            new_string = ''.join(lost_words)
            df.at[idx, 'Category'] = new_string
            df.at[idx, 'Topic'] = split_topic_desc[1]

    else:
        pass

8  Standard
['', 'Flexible Metal Conduit']
8  Standard
['', 'Armored Cable']
8  Standard
['', 'Surface Metal Raceways And Fittings']
0 
8  Standard
['', 'Strut-Type Channel Raceways And Fittings']
8  Standard
['', 'Surface Raceways And Fittings for Use With Data, Signal, And Control Circuits']
0 
8  Standard
['', 'Electrical Rigid Metal Conduit - Aluminum And Stainless Steel']
7  Outline
['', 'Investigation for Pump Tank Extinguishers']
0 
22  Standard Test
['s', 'Window Assemblies']
0 
5  Guide
['', 'Evaluating Doors And Frames for Listing And Labelling']
8  Standard
['', 'Tin-Clad Fire Doors']
22  Standard Test
['s', 'Door Assemblies']
0 
40  Standard Test
['s', 'Door Assemblies']
22  Standard Test
['s', 'Fire-Protective Curtain Assemblies']
8  Standard
['', 'Power-Limited Circuit Cables']
5  Guide
['', 'Hardware for Fire Doors And Emergency Exits']
8  Standard
['', 'Sliding Hardware for Standard, Horizontally Mounted Tin-Clad Fire Doors']
8  Standard
['', 'Swinging Hardware for Stan

In [113]:
df['Topic']

0                                  Flexible Metal Conduit
1                                           Armored Cable
2                     Surface Metal Raceways And Fittings
3               Nonmetallic Surface Raceways And Fittings
4                Strut-Type Channel Raceways And Fittings
                              ...                        
2118    Instruments Used To Detect Oxygen-Deficient/Ox...
2119    Low-Voltage Switchgear And Controlgear- Contro...
2120                                      Office Machines
2121    Ecologo Certification Program for Responsible ...
2122    Ecologo Certification Program for Responsible ...
Name: Topic, Length: 2123, dtype: object

In [120]:
#YAKE
import yake


language = "en"
max_ngram_size = 5
deduplication_threshold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 5

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)
# keywords = custom_kw_extractor.extract_keywords(df['Topic'][0])
# keyword1 = keywords[0][0]
# keyword2 = keywords[1][0]
# keyword3 = keywords[2][0]
# keyword4 = keywords[3][0]
# keyword5 = keywords[4][0]

for idx, row in df.iterrows():
    keywords = custom_kw_extractor.extract_keywords(row['Topic'])
    if len(keywords) < 2:
        df.at[idx, 'Keyword1'] = keywords[0][0]
    elif len(keywords) < 3:
        df.at[idx, 'Keyword1'] = keywords[0][0]
        df.at[idx, 'Keyword2'] = keywords[1][0]
    elif len(keywords) < 4:
        df.at[idx, 'Keyword1'] = keywords[0][0]
        df.at[idx, 'Keyword2'] = keywords[1][0]
        df.at[idx, 'Keyword3'] = keywords[2][0]
    elif len(keywords) < 5:
        df.at[idx, 'Keyword1'] = keywords[0][0]
        df.at[idx, 'Keyword2'] = keywords[1][0]
        df.at[idx, 'Keyword3'] = keywords[2][0]
        df.at[idx, 'Keyword4'] = keywords[3][0]
    else:
        df.at[idx, 'Keyword1'] = keywords[0][0]
        df.at[idx, 'Keyword2'] = keywords[1][0]
        df.at[idx, 'Keyword3'] = keywords[2][0]
        df.at[idx, 'Keyword4'] = keywords[3][0]
        df.at[idx, 'Keyword5'] = keywords[4][0]

df


    # if len(keywords) < 2:
    #     keyword1 = keywords[0][0]
    # else:
    #     keyword1 = ''
    #
    #
    # df.at[idx, 'Keyword1'] = keyword1
    # df.at[idx, 'Keyword2'] = keyword2
    # df.at[idx, 'Keyword3'] = keyword3
    # df.at[idx, 'Keyword4'] = keyword4
    # df.at[idx, 'Keyword5'] = keyword5

# keywords to separate columns




,Standard_Number,Description,Label,Catalog,Edition_Date,Type,English?,Category,Topic,Keyword1,Keyword2,Keyword3,Keyword4,Keyword5
0,1,Standard for Flexible Metal Conduit,Ed. 11,UL,"February 16, 2005",ulstd,True,Standard,Flexible Metal Conduit,Flexible Metal Conduit,Metal Conduit,Flexible Metal,Conduit,Metal
1,4,Standard for Armored Cable,Ed. 15,UL,"January 15, 2004",ulstd,True,Standard,Armored Cable,Armored Cable,Cable,Armored,NaN,NaN
2,5,Standard for Surface Metal Raceways And Fittings,Ed. 15,UL,"May 24, 2016",ulstd,True,Standard,Surface Metal Raceways And Fittings,Surface Metal Raceways And Fittings,Metal Raceways And Fittings,Surface Metal Raceways,Raceways And Fittings,Metal Raceways
3,5A,Nonmetallic Surface Raceways And Fittings,Ed. 4,UL,"June 26, 2015",ulcsa,True,,Nonmetallic Surface Raceways And Fittings,Nonmetallic Surface Raceways And Fittings,Surface Raceways And Fittings,Nonmetallic Surface Raceways,Raceways And Fittings,Surface Raceways
4,5B,Standard for Strut-Type Channel Raceways And F...,Ed. 2,UL,"April 14, 2004",ulstd,True,Standard,Strut-Type Channel Raceways And Fittings,Strut-Type Channel Raceways And Fittings,Channel Raceways And Fittings,Strut-Type Channel Raceways,Raceways And Fittings,Channel Raceways
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2118,920401,Performance Requirements for Instruments Used ...,Ed. 1,UL,"November 15, 2007",ulisa,True,Performance Requirement,Instruments Used To Detect Oxygen-Deficient/Ox...,Instruments Used To Detect Oxygen-Deficient,Oxygen-Enriched Atmospheres,Instruments Used To Detect,Detect Oxygen-Deficient,Atmospheres
2119,62091,Low-Voltage Switchgear And Controlgear- Contro...,Ed. 1,UL,"September 30, 2020",ulcsaiec,True,,Low-Voltage Switchgear And Controlgear- Contro...,Drivers of Stationary Fire Pumps,Controllers for Drivers of Stationary,Stationary Fire Pumps,Low-Voltage Switchgear And Controlgear,Drivers of Stationary Fire
2120,CCD 035,Office Machines,Ed. 1,EcoLogo Standards,"January 01, 2012",NaN,True,,Office Machines,Office Machines,Machines,Office,NaN,NaN
2121,2723,Ecologo Certification Program for Responsible ...,Ed. 1,EcoLogo Standards,"October 09, 2019",NaN,True,,Ecologo Certification Program for Responsible ...,Development for Mineral Exploration Companies,Certification Program for Responsible Development,Responsible Development for Mineral Exploration,Ecologo Certification Program for Responsible,Mineral Exploration Companies


In [121]:
df.to_csv('Results_4.csv', index=False)

In [33]:
# cluster by keywrods


In [4]:
# from top2vec import Top2Vec
# model = Top2Vec(documents=df['Description'].tolist())



KeyboardInterrupt

